In [173]:
import pandas as pd
import datetime as dt
from datetime import  date, timedelta
import numpy as np
import math

In [174]:
demand = pd.read_excel("SCHF round 3.xlsx",sheet_name = 'demand' ,header = None)
display(demand)
T_demand = demand.T
demand_df = T_demand.rename(columns=T_demand.iloc[0]).drop(T_demand.index[0])
# trong b1: đọc các sheet, đối với Sheet Demand thì transpose lại (chuyển cột thành dòng)
ngli =  pd.read_excel("SCHF round 3.xlsx",sheet_name = 'ngli' )
tonkho =  pd.read_excel("SCHF round 3.xlsx",sheet_name = 'tonkho' )
item =  pd.read_excel("SCHF round 3.xlsx",sheet_name = 'item')
dgvc = pd.read_excel("SCHF round 3.xlsx",sheet_name = 'Sheet5')

,0,1,2,3
0,ITEM,APR_2023,MAY_2023,JUN_2023
1,STAR_SHEEP,9515,40750,39250
2,WOW_CHICKEN,66000,90000,72000


In [175]:
for i in range(demand_df.shape[0]):  
    demand_df.iloc[i,0] = dt.datetime.strptime(demand_df.iloc[i,0], '%b_%Y')
demand_df

,ITEM,STAR_SHEEP,WOW_CHICKEN
1,2023-04-01 00:00:00,9515,66000
2,2023-05-01 00:00:00,40750,90000
3,2023-06-01 00:00:00,39250,72000


In [176]:
dgvc

,NGUYÊN LIỆU,MÃ ĐƠN ĐẶT HÀNG,NGÀY VỀ,SỐ LƯỢNG,ĐƠN VỊ,HẠN SỬ DỤNG


In [177]:
thanhphan = ngli['NGUYÊN LIỆU'].unique().tolist()
thanhphan.append('NGÀY SX')
thanhphan[0:]

['NGÔ',
 'KHOAI',
 'SẮN',
 'XƯƠNG_ĐỘNG_VẬT',
 'MEN_VI_SINH',
 'BỘT_ĐỖ_TƯƠNG',
 'THÓC',
 'NGÀY SX']

In [178]:
demand_df = demand_df.rename(columns={'ITEM':'NGÀY SX'})
demand_df.reset_index(drop=True, inplace=True)
demand_df

,NGÀY SX,STAR_SHEEP,WOW_CHICKEN
0,2023-04-01 00:00:00,9515,66000
1,2023-05-01 00:00:00,40750,90000
2,2023-06-01 00:00:00,39250,72000


In [179]:
cacloaithanhpham = ['STAR_SHEEP','WOW_CHICKEN']

demand_detail =pd.DataFrame( 0, index=np.arange(1),columns=thanhphan)
for i in range(demand_df.shape[0]):
    demand_detail.loc[i,['NGÀY SX']] = demand_df.loc[i,['NGÀY SX']]
    demand_detail= demand_detail.fillna(0)
    date = demand_detail.loc[i,['NGÀY SX']][0]
    for j in cacloaithanhpham: 
        for k in ngli[ngli['THÀNH PHẨM'] ==j ]['NGUYÊN LIỆU'].tolist():
            soluongtrongthang = demand_df[demand_df['NGÀY SX']==demand_detail.loc[i,['NGÀY SX']][0]][j].tolist()[0]
            soluongcanchomotsanpham = ngli.loc[(ngli['THÀNH PHẨM']== j )  & (ngli['NGUYÊN LIỆU']== k ) ,'SỐ LƯỢNG NGUYÊN LIỆU TRÊN 1 THÀNH PHẨM'].tolist()[0]
            demand_detail.loc[demand_detail['NGÀY SX']== date,k] = demand_detail.loc[demand_detail['NGÀY SX']== date,k]+soluongtrongthang*soluongcanchomotsanpham


In [180]:
import copy
demand_detail_show = copy.copy(demand_detail)
demand_detail_show['NGÀY SX'] = demand_detail_show['NGÀY SX'].apply(lambda x: str(x)[:7])
demand_detail_show

,NGÔ,KHOAI,SẮN,XƯƠNG_ĐỘNG_VẬT,MEN_VI_SINH,BỘT_ĐỖ_TƯƠNG,THÓC,NGÀY SX
0,18878.75,4757.5,1189.375,9439.375,1887.875,8250.0,33000.0,2023-04
1,32687.50,20375.0,5093.750,16343.750,3268.750,11250.0,45000.0,2023-05
2,27812.50,19625.0,4906.250,13906.250,2781.250,9000.0,36000.0,2023-06


In [181]:
item.loc[tonkho.ITEM == 'BAO_BÌ_ĐÓNG_GÓI','SHELF LIFE ĐƯỢC ƯỚC TÍNH KỂ TỪ NGÀY NHẬP KHO (NGÀY)'] = 300000


In [182]:
demand_detail

,NGÔ,KHOAI,SẮN,XƯƠNG_ĐỘNG_VẬT,MEN_VI_SINH,BỘT_ĐỖ_TƯƠNG,THÓC,NGÀY SX
0,18878.75,4757.5,1189.375,9439.375,1887.875,8250.0,33000.0,2023-04-01
1,32687.50,20375.0,5093.750,16343.750,3268.750,11250.0,45000.0,2023-05-01
2,27812.50,19625.0,4906.250,13906.250,2781.250,9000.0,36000.0,2023-06-01


In [183]:
hangdangdat = pd.DataFrame( 0, index=np.arange(1),columns=['ITEM','SOLUONG','NHACUNGCAP','NGAYDATHANG'])
minday = dt.datetime(2023, 1, 1 ) # ngày đặt hàng min 
# dòng phía trên để tạo bảng mới
for i in range(demand_df.shape[0]): # chạy qua các tháng 1,2,3,4,5,6
    tonkho.loc[tonkho.ITEM == 'BAO_BÌ_ĐÓNG_GÓI','HẠN SỬ DỤNG'] = dt.datetime(2090, 5, 17)
    #chỉnh lại hạn sử dụng của bao bì cho đồng bộ vs cái ở phía trên
    tonkho =  tonkho.loc[~(tonkho==0).all(axis=1)]
    #loại bỏ những dòng có giá trị toàn là 0 (vì đã được sử dụng hết)
    tonkho.reset_index(drop=True, inplace=True)
    tonkho  = tonkho.sort_values('HẠN SỬ DỤNG',ignore_index=True )
    # sort value theo hạn sử dụng để đáp ứng nguyên tắc FEFO
    month = demand_df.iloc[i,0].year*100+demand_df.iloc[i,0].month 
    # month là biến được tính theo tháng: ví dụ ngày 19/2/2022 so  với ngày 20/3/2022 sẽ được chuyển thành 202202 và 202203 thì
    # 202202 sẽ bé hơn 202203
    # Vì dữ liệu cho trong đề là lead time hàng sản xuất là 0 ngày nên ta chỉ cần xét đến tháng là được,không cần xét ngày
    getstock = 0
    
    for tonkh in range(tonkho.shape[0]):   
        try:
            if (tonkho.loc[tonkh,['HẠN SỬ DỤNG']][0].year*100 + tonkho.loc[tonkh,['HẠN SỬ DỤNG']][0].month) < month: 
                print('Het han su dung: ','Item:',tonkho.iloc[tonkh,0],'    So luong:',tonkho.iloc[tonkh,1]  ,'   Ngay het han:',tonkho.iloc[tonkh,3])
                  # câu lệnh print sẽ in bên dưới  het hạn sử dụng
                for zerofill in tonkho.columns:
                    tonkho.loc[tonkh,zerofill] = 0
            # trong đoạn try đến except này dùng để loại những hàng hết hạn sử dụng, những hàng đó được đưa giá trị về 0
        except:
            pass
    tonkho = tonkho.loc[~(tonkho==0).all(axis=1)]
    tonkho.reset_index(drop=True, inplace=True)
    # 2 dòng trên dùng để loại những hàng có giá trị toàn là 0, cũng là loại những hàng hết hạn sử dụng
    dgvc =  dgvc.loc[~(dgvc==0).all(axis=1)]
    for dgv in range(dgvc.shape[0]):
        appendlist=[0,0,0,0]
        if (dgvc.loc[dgv,['NGÀY VỀ']][0].year*100 + dgvc.loc[dgv,['NGÀY VỀ']][0].month) == month: 
            appendlist=[dgvc.iloc[dgv,0],dgvc.iloc[dgv,3],dgvc.iloc[dgv,4],dgvc.iloc[dgv,5]]
            tonkho.loc[len(tonkho)] =appendlist
            for zerofill in dgvc.columns:
                    dgvc.loc[dgv,zerofill] = 0      
            # những hàng nào mà ngày về có tháng trùng với tháng đang chạy vòng lặp thì được đưa vào hàng tồn kho, các dữ liệu 
            #được đưa vào gồm hạn sử dụng, tên item, số luọngw
    dgvc =  dgvc.loc[~(dgvc==0).all(axis=1)]
    dgvc.reset_index(drop=True, inplace=True)
    hetngli = 0
    for j in thanhphan[0:-1]:
        hetngli = 0
        if demand_detail.loc[i,j] > 0 and hetngli == 0:
            try:
                drop_index = tonkho.loc[tonkho.ITEM==j,:].index.tolist()[0]
                soluongtonkho = tonkho.loc[drop_index,"TỒN KHO CUỐI NGÀY 31/3/2023"]
                tongtonkho = tonkho.loc[tonkho.ITEM==j,:]['TỒN KHO CUỐI NGÀY 31/3/2023'].sum()
            except:
                tongtonkho = 0
            # câu lệnh trên xét xem trong kho còn bao nhiêu, các item trong tonkho đã đc sắp xếp theo FEFO
            # từ ĐÂY
            if demand_detail.loc[i,j] <= tongtonkho:
            
                while demand_detail.loc[i,j]>0:
                    if demand_detail.loc[i,j] >= soluongtonkho:
                        demand_detail.loc[i,j] -=soluongtonkho
                        tonkho.drop(index = drop_index, inplace = True)
                        tonkho.reset_index(drop=True, inplace=True)
                        drop_index = tonkho.loc[tonkho.ITEM==j,:].index.tolist()[0]
                        soluongtonkho = tonkho.loc[drop_index,"TỒN KHO CUỐI NGÀY 31/3/2023"]
                    if demand_detail.loc[i,j] < soluongtonkho:
                        tonkho.loc[drop_index,"TỒN KHO CUỐI NGÀY 31/3/2023"] -= demand_detail.loc[i,j]
                        demand_detail.loc[i,j] = 0
                    # đến ĐÂY là xét điều kiện
            # nếu số lượng tồn kho lớn hơn số nguyên liệu yêu cầu trong tháng thì không cần đặt hàng, hàng trong kho
            # sẽ giảm đi số lượng tương ứng 
            # còn phần ELSE trở về dưới là khi mà số lượng trong kho không đáp ứng đủ, phải đặt hàng mới
            else:
                soluongmoi = demand_detail.loc[i,j] - tongtonkho
                tongdathang = max(soluongmoi,item.loc[item.ITEM == j,'MOQ'].tolist()[0])
                # câu lệnh trên xét Min order Quantity
                shelflife = item.loc[item.ITEM == j,'Shelf life'].tolist()[0]
                # tính Shelflife để xét hạn sử dụng
                
                hansudungcuahang =  demand_df.iloc[i,0]+  timedelta(days = shelflife)
                #câu tính shelf life đó sai nên cần phải tính theo công thức bên dưới
                lead = item.loc[item.ITEM == j,'LEADTIME ĐẶT HÀNG (NGÀY)'].tolist()[0]
                #tính lead time để tính ngày đặt hàng
                ngaydathang =  demand_df.iloc[i,0]-  timedelta(days = 7)
                hansudungcuahang = ngaydathang +  timedelta(days = shelflife) + timedelta(days = lead)
                
                if ngaydathang< minday:
                    ngaydathang =  minday
                    hansudungcuahang =  minday +  timedelta(days = shelflife)+timedelta(days = lead)
                    
                
                ncc = item.loc[item.ITEM == j,'NHÀ CUNG CẤP'].tolist()[0]
                hangthua = tongdathang-demand_detail.loc[i,j]
                # tính số nguyên liệu còn thừa lại sau khi sử dụng, những nguyên liệu còn thừa được đưa vào bảng tonkho
                demand_detail.loc[i,j] = 0
            
                try:
                    tonkho.loc[tonkho.ITEM==j,:]= 0
                    tonkho =  tonkho.loc[~(tonkho==0).all(axis=1)]
                    tonkho.reset_index(drop=True, inplace=True)
                except:
                    pass
                if hangthua >0:
                    
                    appendlist3 = [j,hangthua,'kg',hansudungcuahang]
                    print(appendlist3)
                    print(tonkho)
                    tonkho.loc[len(tonkho)] =appendlist3
                appendlist = [j,tongdathang,ncc,ngaydathang]
                hangdangdat.loc[len(hangdangdat)] =appendlist
                # thêm vào bảng hàng đang đặt những hàng cần thiết phải đặt
        

Het han su dung:  Item: SẮN     So luong: 809.625    Ngay het han: 2023-04-23 00:00:00
Het han su dung:  Item: KHOAI     So luong: 2437.5    Ngay het han: 2023-04-30 00:00:00


In [184]:
hangdangdat =  hangdangdat.loc[~(hangdangdat==0).all(axis=1)]
hangdangdat.reset_index(drop=True, inplace=True)
hangdangdat1= copy.copy(hangdangdat)
hangdangdat1.NGAYDATHANG = hangdangdat1.NGAYDATHANG.apply(lambda x: str(x)[0:7])
hangdangdat1.SOLUONG = hangdangdat1.SOLUONG*1.02

In [185]:
hangdangdat1.pivot(index = ['ITEM','NHACUNGCAP'],columns = ['NGAYDATHANG']).fillna(0)

# pivot lại để tạo ra format cuối cùng

SOLUONG                      
NGAYDATHANG                           2023-03    2023-04    2023-05
ITEM           NHACUNGCAP                                          
BỘT_ĐỖ_TƯƠNG   CÔNG TY TNHH NARGI   4521.6600  11475.000   9180.000
KHOAI          CÔNG TY TNHH NARGI      0.0000  20782.500  20017.500
MEN_VI_SINH    CÔNG TY VNIBIO       1082.0925   3334.125   2836.875
NGÔ            CÔNG TY TNHH NARGI  10544.5050  33341.250  28368.750
SẮN            CÔNG TY TNHH NARGI      0.0000   5195.625   5004.375
THÓC           CÔNG TY TNHH NARGI  22314.5400  45900.000  36720.000
XƯƠNG_ĐỘNG_VẬT CÔNG TY CP YFOOD     4322.1225  16670.625  14184.375

In [186]:
demand_detail_show

,NGÔ,KHOAI,SẮN,XƯƠNG_ĐỘNG_VẬT,MEN_VI_SINH,BỘT_ĐỖ_TƯƠNG,THÓC,NGÀY SX
0,18878.75,4757.5,1189.375,9439.375,1887.875,8250.0,33000.0,2023-04
1,32687.50,20375.0,5093.750,16343.750,3268.750,11250.0,45000.0,2023-05
2,27812.50,19625.0,4906.250,13906.250,2781.250,9000.0,36000.0,2023-06


In [187]:
month

202306

In [188]:
demand_df

,NGÀY SX,STAR_SHEEP,WOW_CHICKEN
0,2023-04-01 00:00:00,9515,66000
1,2023-05-01 00:00:00,40750,90000
2,2023-06-01 00:00:00,39250,72000
